In [1]:
import os

In [2]:
%pwd

'd:\\Data Science\\Git-Upload-Projects\\End_to_End_winequality\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Data Science\\Git-Upload-Projects\\End_to_End_winequality'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/singh.shanu1988/End_to_End_winequality.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="singh.shanu1988"
os.environ["MLFLOW_TRACKING_PASSWORD"]="85d3b3428bfdf23ee7d518a37c50b190e4e54914"


# MLFLOW_TRACKING_URI=https://dagshub.com/singh.shanu1988/End_to_End_winequality.mlflow \
# MLFLOW_TRACKING_USERNAME=singh.shanu1988 \
# MLFLOW_TRACKING_PASSWORD=85d3b3428bfdf23ee7d518a37c50b190e4e54914 \
# python script.py


In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str


In [7]:
from winequality.constants import *
from winequality.utils.common import read_yaml, create_directories, save_json

In [8]:
class ConfigurationManager:
     
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schemas_filepath = SCHEMA_FILE_PATH):
        
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)
            self.schema = read_yaml(schemas_filepath)

            create_directories([self.config.dataStore_root])
        

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/singh.shanu1988/End_to_End_winequality.mlflow",
        )
        return model_evaluation_config

In [9]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [12]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
    
    def eval_metrics(self, actual, pred):
        r2 = r2_score(actual, pred)
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        return r2, rmse, mae
    
    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        x_test = test_data.drop([self.config.target_column], axis=1)
        y_test = test_data[self.config.target_column]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_artifact_uri()).scheme

        with mlflow.start_run():
            predict_quality = model.predict(x_test)
            (r2, rmse, mae) = self.eval_metrics(y_test, predict_quality)

            # Saving metrics as local
            scores = {"r2":r2, "rmse":rmse, "mae":mae}
            save_json(path=Path(self.config.metric_file_name), data=scores)
            
            mlflow.log_params(self.config.all_params)
            
            mlflow.log_metrics("rmse", rmse)
            mlflow.log_metrics("r2", r2)
            mlflow.log_metrics("mae", mae)
            
            # Model registry does not work with file store
            if tracking_url_type_store != "file":
                
                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model,"model", registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model")

In [13]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e


[2024-04-09 23:15:55,276: INFO: common: YAML file loaded successfully: config\config.yaml]
[2024-04-09 23:15:55,278: INFO: common: YAML file loaded successfully: params.yaml]
[2024-04-09 23:15:55,278: INFO: common: YAML file loaded successfully: schema.yaml]
[2024-04-09 23:15:55,287: INFO: common: Created directory at: dataStore]
[2024-04-09 23:15:55,289: INFO: common: Created directory at: dataStore/model_evaluation]


Exception: Run with UUID 05d3544b90be42509f173ed93bad5fe5 is already active. To start a new run, first end the current run with mlflow.end_run(). To start a nested run, call start_run with nested=True